In [1]:
import pandas as pd
import scipy.optimize as opt
import numpy as np

In [62]:
# read in demand data
demand = pd.read_csv("/content/apples_demand.csv")

# read in supply predictions data
actual_supply = pd.read_csv("/content/apples_var_predict.csv")

# read in prices data
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [63]:
demand = demand[demand["year"] >= 1998]
demand = demand[demand["year"] <= 2015]
demand.reset_index(inplace=True, drop=True)

demand.head()

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,1.196802e+08,5.505405e+07,4.087547e+07,5.105357e+07,2.796828e+08,5.674611e+07,3.834188e+07,2.844886e+07,6.298862e+07,4.932338e+07,5.188800e+07,6.972608e+07,1.469968e+08,8.254484e+07,3.763447e+07,4.438299e+07,7.149224e+07
1,1999,1.506239e+08,7.356510e+07,5.376353e+07,7.799250e+07,3.568051e+08,8.708944e+07,5.596149e+07,4.301247e+07,9.852386e+07,7.267205e+07,7.949703e+07,9.324799e+07,2.077520e+08,1.095237e+08,4.986791e+07,6.528588e+07,9.854023e+07
2,2000,1.111720e+08,6.183426e+07,4.628281e+07,6.186938e+07,3.271555e+08,8.334606e+07,4.767419e+07,3.392062e+07,9.102741e+07,6.062981e+07,7.570757e+07,7.919431e+07,1.782399e+08,8.824238e+07,3.596253e+07,6.283158e+07,8.500531e+07
3,2001,1.602234e+08,7.432132e+07,5.953028e+07,8.130428e+07,4.165989e+08,1.065772e+08,5.786207e+07,4.069766e+07,1.165657e+08,7.372698e+07,9.787429e+07,9.140943e+07,2.131316e+08,1.032571e+08,4.034850e+07,7.904971e+07,1.066164e+08
4,2002,1.596045e+08,7.120223e+07,6.173363e+07,8.704060e+07,3.984746e+08,8.786359e+07,6.723919e+07,4.567788e+07,1.063752e+08,7.421841e+07,9.682131e+07,9.714415e+07,2.243621e+08,9.592922e+07,4.476036e+07,7.380568e+07,1.054266e+08


In [64]:
actual_supply = actual_supply[actual_supply["year"] >= 1999]
actual_supply.reset_index(inplace=True, drop=True)

actual_supply.head()

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,9.228897e+08,2.891977e+07,3.349362e+08,5.604570e+08,9.518095e+08,8.953932e+08
1,2000,6.997051e+08,2.191886e+07,2.489601e+08,4.139080e+08,7.216240e+08,6.628680e+08
2,2001,8.820604e+08,2.768480e+07,3.117965e+08,5.172664e+08,9.097452e+08,8.290629e+08
3,2002,7.030500e+08,2.214044e+07,2.459096e+08,4.065423e+08,7.251904e+08,6.524519e+08
4,2003,6.810910e+08,2.156555e+07,2.358502e+08,3.886300e+08,7.026566e+08,6.244802e+08


In [65]:
wholesale_prices = wholesale_prices[wholesale_prices["year"] >= 1999]
wholesale_prices = wholesale_prices[wholesale_prices["year"] <= 2015]
wholesale_prices.reset_index(inplace=True, drop=True)

wholesale_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,21.3,35.0,33.00,28.6,74.4,102,17.250,11.101633,67.161655,...,16.8,24.1,32.1,15.098916,46.5,11.3,9.7,16.9,27.428610,6.9
1,2000,17.8,36.0,32.35,27.6,64.9,115,7.000,11.035726,79.001992,...,13.1,31.2,31.8,19.228349,42.6,12.3,11.2,18.5,28.129329,5.3
2,2001,22.9,38.0,34.50,30.7,75.8,109,9.625,11.863701,72.454395,...,17.1,26.5,32.4,19.287165,45.0,13.4,10.7,19.5,27.847922,10.8
3,2002,25.8,43.0,34.30,30.6,71.3,117,11.250,12.184179,87.104935,...,19.1,31.4,34.2,23.559195,47.6,12.9,12.1,19.2,27.546230,9.6
4,2003,26.1,41.0,34.45,29.1,74.9,120,8.125,12.388878,79.634487,...,19.0,32.7,36.9,21.827844,49.2,13.3,13.7,19.2,28.384431,7.3


In [66]:
retail_prices = retail_prices[retail_prices["year"] >= 1999]
retail_prices = retail_prices[retail_prices["year"] <= 2015]
retail_prices.reset_index(inplace=True, drop=True)

retail_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,86.0,52.5,49.500,42.90,208,153.0,25.8750,16.652449,100.742482,...,25.20,36.15,48.15,22.648374,69.75,16.95,14.55,25.35,41.142914,10.35
1,2000,88.0,54.0,48.525,41.40,188,172.5,10.5000,16.553589,118.502987,...,19.65,46.80,47.70,28.842524,63.90,18.45,16.80,27.75,42.193994,7.95
2,2001,83.0,57.0,51.750,46.05,229,163.5,14.4375,17.795551,108.681593,...,25.65,39.75,48.60,28.930747,67.50,20.10,16.05,29.25,41.771883,16.20
3,2002,91.0,64.5,51.450,45.90,228,175.5,16.8750,18.276268,130.657402,...,28.65,47.10,51.30,35.338792,71.40,19.35,18.15,28.80,41.319345,14.40
4,2003,94.0,61.5,51.675,43.65,237,180.0,12.1875,18.583316,119.451730,...,28.50,49.05,55.35,32.741766,73.80,19.95,20.55,28.80,42.576646,10.95


In [67]:
# parameters
N = 16 # number of retailers
K = demand.shape[0]-1 # number of iterations (0th one is the initial condition)

# dataframes to keep track of orders
# columns represent retailers, rows represent iterations
orders_west = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_p_sw = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_mw_se = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_a_ne = pd.DataFrame(columns=range(0,N), index=range(0,K+1))

# initial orders
for i in range(0,N):
  orders_west.iloc[0,i] = demand["Average"].iloc[0] / 4
  orders_p_sw.iloc[0,i] = demand["Average"].iloc[0] / 4
  orders_mw_se.iloc[0,i] = demand["Average"].iloc[0] / 4
  orders_a_ne.iloc[0,i] = demand["Average"].iloc[0] / 4

In [68]:
def ret_cost_sim(q, k, curr_i, orders_west, orders_p_sw, orders_mw_se, orders_a_ne, demand, supply, wholesale_prices, retail_prices):
  Q_i = demand.iloc[k+1,curr_i+1]

  total_request_west = q[0]
  total_request_west += orders_west.iloc[k,0:curr_i].sum() + orders_west.iloc[k,curr_i+1:].sum()

  total_request_p_sw = q[1]
  total_request_p_sw += orders_p_sw.iloc[k,0:curr_i].sum() + orders_p_sw.iloc[k,curr_i+1:].sum()

  total_request_mw_se = q[2]
  total_request_mw_se += orders_mw_se.iloc[k,0:curr_i].sum() + orders_mw_se.iloc[k,curr_i+1:].sum()

  total_request_a_ne = q[3]
  total_request_a_ne += orders_a_ne.iloc[k,0:curr_i].sum() + orders_a_ne.iloc[k,curr_i+1:].sum()

  if (total_request_west <= supply["west_lbs"][k]):
    curr_R_west = q[0]
  else:
    curr_R_west = supply["west_lbs"][k] * (q[0] / total_request_west)

  if (total_request_p_sw <= supply["p_sw_lbs"][k]):
    curr_R_p_sw = q[1]
  else:
    curr_R_p_sw = supply["p_sw_lbs"][k] * (q[1] / total_request_p_sw)

  if (total_request_mw_se <= supply["mw_se_lbs"][k]):
    curr_R_mw_se = q[2]
  else:
    curr_R_mw_se = supply["mw_se_lbs"][k] * (q[2] / total_request_mw_se)

  if (total_request_a_ne <= supply["a_ne_lbs"][k]):
    curr_R_a_ne = q[3]
  else:
    curr_R_a_ne = supply["a_ne_lbs"][k] * (q[3] / total_request_a_ne)

  R = curr_R_west + curr_R_p_sw + curr_R_mw_se + curr_R_a_ne

  holding = max(R - Q_i, 0)
  stockout = max(Q_i - R, 0)

  holding = wholesale_prices["Apples"][k] * holding
  stockout = retail_prices["Apples"][k] * stockout

  cost = holding + stockout

  return cost

In [ ]:
for k in range(0,K):
  print(k)

  for curr_i in range(0,N):
    rranges = (slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5))
    params_curr = (k, curr_i, orders_west, orders_p_sw, orders_mw_se, orders_a_ne, demand, actual_supply, wholesale_prices, retail_prices)
    opt_sol = opt.brute(ret_cost_sim, rranges, args=params_curr, full_output=True)

    if opt_sol[0][0] > 0:
      orders_west.iloc[k+1,curr_i] = opt_sol[0][0]
    else:
      orders_west.iloc[k+1,curr_i] = 0

    if opt_sol[0][1] > 0:
      orders_p_sw.iloc[k+1,curr_i] = opt_sol[0][1]
    else:
      orders_p_sw.iloc[k+1,curr_i] = 0

    if opt_sol[0][2] > 0:
      orders_mw_se.iloc[k+1,curr_i] = opt_sol[0][2]
    else:
      orders_mw_se.iloc[k+1,curr_i] = 0

    if opt_sol[0][3] > 0:
      orders_a_ne.iloc[k+1,curr_i] = opt_sol[0][3]
    else:
      orders_a_ne.iloc[k+1,curr_i] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
actual_supply

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,8.112974e+08,2.541931e+07,2.919481e+08,4.871825e+08,8.367167e+08,7.791306e+08
1,2000,9.201658e+08,2.886866e+07,3.283285e+08,5.463827e+08,9.490344e+08,8.747111e+08
2,2001,7.044119e+08,2.216599e+07,2.492134e+08,4.135795e+08,7.265779e+08,6.627929e+08
3,2002,7.439219e+08,2.352332e+07,2.609339e+08,4.318310e+08,7.674452e+08,6.927648e+08
4,2003,8.052488e+08,2.563933e+07,2.799623e+08,4.620355e+08,8.308881e+08,7.419977e+08
5,2004,1.043176e+09,3.351982e+07,3.593539e+08,5.914081e+08,1.076696e+09,9.507620e+08
6,2005,8.592295e+08,2.792639e+07,2.930679e+08,4.809710e+08,8.871559e+08,7.740390e+08
7,2006,8.865187e+08,2.920874e+07,3.031627e+08,4.863157e+08,9.157274e+08,7.894784e+08
8,2007,8.745792e+08,2.926458e+07,3.008015e+08,4.720713e+08,9.038438e+08,7.728728e+08
9,2008,8.819580e+08,2.833992e+07,3.092961e+08,4.778698e+08,9.102979e+08,7.871658e+08


In [60]:
orders_total = orders_west + orders_p_sw + orders_mw_se + orders_a_ne
(orders_total.iloc[:,2] - demand["Minneapolis.StPaul"]) / 10**7

0      3.061677
1      0.149939
2      0.917179
3      0.289954
4      0.877917
5       1.92021
6      2.357247
7      1.616698
8      2.854892
9      4.648005
10    13.591315
11     2.769871
12     7.687509
13     2.518679
14     7.443276
15     1.730798
16     5.108816
17     3.137408
dtype: object

In [38]:
orders_total = orders_west + orders_p_sw + orders_mw_se + orders_a_ne
(orders_total.iloc[:,3] - demand["Boston"]) / 10**7

0      2.043867
1      0.231168
2      3.011466
3      0.375342
4      3.149265
5      2.690697
6      6.146863
7      1.995045
8      5.617455
9      3.698075
10    19.126714
11     4.131183
12     2.475898
13     7.031603
14    17.533563
15     1.508208
16      9.20668
17     4.843434
dtype: object

In [14]:
demand

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,1.196802e+08,5.505405e+07,4.087547e+07,5.105357e+07,2.796828e+08,5.674611e+07,3.834188e+07,2.844886e+07,6.298862e+07,4.932338e+07,5.188800e+07,6.972608e+07,1.469968e+08,8.254484e+07,3.763447e+07,4.438299e+07,7.149224e+07
1,1999,1.506239e+08,7.356510e+07,5.376353e+07,7.799250e+07,3.568051e+08,8.708944e+07,5.596149e+07,4.301247e+07,9.852386e+07,7.267205e+07,7.949703e+07,9.324799e+07,2.077520e+08,1.095237e+08,4.986791e+07,6.528588e+07,9.854023e+07
2,2000,1.111720e+08,6.183426e+07,4.628281e+07,6.186938e+07,3.271555e+08,8.334606e+07,4.767419e+07,3.392062e+07,9.102741e+07,6.062981e+07,7.570757e+07,7.919431e+07,1.782399e+08,8.824238e+07,3.596253e+07,6.283158e+07,8.500531e+07
3,2001,1.602234e+08,7.432132e+07,5.953028e+07,8.130428e+07,4.165989e+08,1.065772e+08,5.786207e+07,4.069766e+07,1.165657e+08,7.372698e+07,9.787429e+07,9.140943e+07,2.131316e+08,1.032571e+08,4.034850e+07,7.904971e+07,1.066164e+08
4,2002,1.596045e+08,7.120223e+07,6.173363e+07,8.704060e+07,3.984746e+08,8.786359e+07,6.723919e+07,4.567788e+07,1.063752e+08,7.421841e+07,9.682131e+07,9.714415e+07,2.243621e+08,9.592922e+07,4.476036e+07,7.380568e+07,1.054266e+08
5,2003,1.816112e+08,8.238934e+07,6.810175e+07,9.563877e+07,4.450321e+08,9.211918e+07,8.366273e+07,5.076049e+07,1.132586e+08,8.128128e+07,1.259022e+08,1.103929e+08,2.949533e+08,1.097756e+08,6.050108e+07,8.584059e+07,1.224248e+08
6,2004,1.202226e+08,5.162126e+07,4.640732e+07,6.835816e+07,3.040968e+08,6.862509e+07,5.638226e+07,3.208932e+07,6.998764e+07,5.294254e+07,9.473489e+07,7.479795e+07,2.029465e+08,7.255139e+07,4.621428e+07,5.425910e+07,8.330806e+07
7,2005,1.490393e+08,7.148228e+07,5.697280e+07,8.662137e+07,3.752511e+08,9.772093e+07,6.571178e+07,3.532902e+07,8.116785e+07,6.882877e+07,1.050693e+08,8.659634e+07,2.254615e+08,8.427553e+07,4.697734e+07,6.352473e+07,1.000018e+08
8,2006,1.772735e+08,8.078436e+07,6.379839e+07,9.111231e+07,4.201975e+08,1.129437e+08,6.524460e+07,3.861520e+07,9.355129e+07,8.488679e+07,1.178814e+08,1.022283e+08,2.593130e+08,1.039460e+08,4.476888e+07,6.998902e+07,1.133255e+08
9,2007,3.056596e+08,1.382479e+08,1.234701e+08,1.516682e+08,6.589054e+08,1.635471e+08,1.253122e+08,7.298715e+07,1.554051e+08,1.487003e+08,2.303526e+08,1.928957e+08,4.733807e+08,1.847837e+08,6.967871e+07,1.061628e+08,1.941857e+08


In [61]:
orders_west.to_csv("/content/apples_arma_orders_west.csv", index=False)
orders_p_sw.to_csv("/content/apples_arma_orders_p_sw.csv", index=False)
orders_mw_se.to_csv("/content/apples_arma_orders_mw_se.csv", index=False)
orders_a_ne.to_csv("/content/apples_arma_orders_a_ne.csv", index=False)